# Password Authentication Using Python

In this lab activity, you will explore the logic of how Unix-based systems generate secure passwords by using the Python programming language.

# Importing the Necessary Modules

To do this activity, you need to have the following python modules installed in your computer:

* passlib

Remember that you can use the command "!pip install ..." to install these modules.

In [5]:
!pip install passlib


Now import the installed modules

In [6]:
import passlib

# Creating a Unix Password

The **passlib.hash** module contains all the password hash algoriths that have been used in the history of UNIX systems, such as DES, MD5 and SHA-256. To use them, ypu only have to import the hashing function and use it with the correct parameters.

If you want to find out more about the currently active and inactive hashes in UNIX, check the following link: https://passlib.readthedocs.io/en/stable/lib/passlib.hash.html

## DES

First, we wil test the classical DES hashing function. To do so, you need to import from the **passlib.hash** module the function called **des_crypt**:

* from passlib.hash import des_crypt

Afterwards, you can call the hashing function by performing the following command:

* deshash1 = des_crypt.hash(password)

In [7]:
## Use this cell to import the DES hashing function and hash the password "a". Store the result in variable 'deshash1'
## and print your result.
from passlib.hash import des_crypt
deshash1 = des_crypt.hash("a")
deshash1

'WXh6Q1mu4pw6s'

Notice that if you call the function again to hash the same password, the output will be different

In [8]:
deshash2 = des_crypt.hash("a")
deshash2

'OGHN6kzJ6Iv5k'

This is due to the fact that the algorithm has added **salt** and thus, it generates a different hash every time. You can specify the salt used by adding the salt keyword to the function as follows:

* deshash3 = des_crypt.hash('abc', salt=**Two characters**)

Where the two characters used can be numbers 0-9, letters a-z;A-Z and special characters ./

In practice, it is better **NOT** to specify the salt, because the aim is to obtain a random hash.

In [9]:
## Use this cell to call the DES hashing function to hash password "a" with the hash "08".
## Store the result in variable 'deshash3' and print your result.
deshash3 = des_crypt.hash("a", salt="08")
deshash3

'0884v.jRPyncg'

You should obtain the hash **0884v.jRPyncg**. Notice that although salt was added, all passwords have the same length:

In [10]:
print('Size of deshash1:', len(deshash1))
print('Size of deshash2:', len(deshash2))
print('Size of deshash3:', len(deshash3))

Size of deshash1: 13
Size of deshash2: 13
Size of deshash3: 13


### Verifying a password

To validate an incoming password against your existing hashes, you can use the verify function.

* des_crypt.verify(**incoming password**, **hashed password**)

In [11]:
## Use this cell to verify different combinations of passwords against your three hashes
des_crypt.verify('a', deshash1)



True

In [12]:

des_crypt.verify('b', deshash2)

False

In [13]:
des_crypt.verify('a', deshash3)

True

### Why  is DES considered insecure?

DES is no longer considered secure, for a variety of reasons:

* Its use of the DES stream cipher, which is vulnerable to practical pre-image attacks, and considered broken, as well as having too-small key and block sizes.
* The 12-bit salt is considered to small to defeat rainbow-table attacks (most modern algorithms provide at least a 48-bit salt).
* The fact that it only uses the lower 7 bits of the first 8 bytes of the password results in a dangerously small keyspace which needs to be searched.

Source: https://passlib.readthedocs.io/en/stable/lib/passlib.hash.des_crypt.html

## Other hashing functions

### MD5

MD5 is more secure than DES and contains an option to specify the size of the salt. This is required by some standards such as Cisco compatible hashes, which require a salt size of four.

In [14]:
from passlib.hash import md5_crypt
md5hash = md5_crypt.hash("a", salt_size = 4)
md5hash

'$1$TZmh$1Wbtm95n4Jcb9yWE9jmOf.'

This hash appears to be more secure than the previous ones!

### Why is MD5 considered insecure?

* It relies on the MD5 message digest, for which theoretical pre-image attacks exist.
* Its fixed number of rounds (combined with the availability of high-throughput MD5 implementations) means this algorithm is increasingly vulnerable to brute force attacks. 

Source: https://passlib.readthedocs.io/en/stable/lib/passlib.hash.md5_crypt.html

## SHA-256

This algorithm includes fixes and advancements such as number of rounds and the use of cryptographic primitives, which are low level functions that further secure the password. In this case, the salt can be from 0 to 16 characters long.

In [15]:
from passlib.hash import sha256_crypt

shahash1 = sha256_crypt.hash("a")
print(shahash1)

# same, but with explict number of rounds
shahash2 = sha256_crypt.using(rounds=12345).hash("abc")
print(shahash2)

# same, but with explict salt
shahash3 = sha256_crypt.hash("abc", salt = 'h84dK.')
print(shahash3)

$5$rounds=535000$ge8HbmwwxNUBSgow$.VFO5cNRmARwIEdTkS2N8GzSaJG6MFC3Uzw6tKCY0M6
$5$rounds=12345$JTzu90Sv4Ygie2np$dycx7WUorZMWjZ4F4vjNxk3v151kQQ0cZ9v9gPeEzh8
$5$rounds=535000$h84dK.$eTIcudqFuA7bfMQza61WMQ73WtvkLmjbfsdOnKaSRj/


Notice that the hash specifies that SHA256 has been used (indicated by the 5) and the number of rounds used.

### Why is SHA-256 considered insecured?

Even SHA256 has some security issues:

* The algorithm’s initialization stage contains a loop which varies linearly with the square of the password size; and further loops, which vary linearly as $password size * rounds$.
* This means an attacker could provide a maliciously large password at the login screen to attempt a DOS on a publically visible login. For example, a 32kb password would require hashing 1Gb of data. Passlib mitigates this by limiting the maximum password size to 4k by default.
* An attacker could also theoretically determine a password’s size by observing the time taken on a successful login, and then attempting verification themselves to find the size password which has an equivalent delay. This has not been applied in practice, probably due to the fact that (for normal passwords < 64 bytes), the contribution of the password size to the overall time taken is below the observable noise level when evesdropping on the timings of successful logins for a single user.

Source: https://passlib.readthedocs.io/en/stable/lib/passlib.hash.sha256_crypt.html

## Cracking DES hashes

A very intuitive way to crack a hash is by doing a brute force attack. This would consist mainly in exhaustively trying combinations of passwords and salts until a match is achieved.

To implement a brute force attack for one-character long passwords, you can use the following code:

In [16]:
# This is a function that performs a brute force attack on DES hashe with salt (i.e. deshash3)
# In this scenario, you have the advantage of knowing that the password to break is one character long
import time

# 1. Declare strings "list_of_saltchars" and "list_of_passchars" (2 lines)
list_of_saltchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./"
list_of_passchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./\-+%@,*"
def bruteForce(hashtobreak):
# 2. Iterate the list of salt chars using two fors so that you can test all possible salt combinations
    for i in list_of_saltchars:
        for j in list_of_saltchars:
            salt = i+j
            # 3. For all characters in list_of_passchars, hash with the salt
            for k in list_of_passchars:
                attempt = des_crypt.hash(k, salt=salt)
                # 4. If there is a match, exit and return the charcter
                if attempt == hashtobreak:
                    # 5. Return the broken password and the broken salt
                    return k, salt
     
t = time.time()
brokenhash, brokensalt = bruteForce(deshash3)
print('The password is:', brokenhash)
print('The salt is:', brokensalt)
print('Elapsed time to break the hash: ', time.time() - t)

The password is: a
The salt is: 08
Elapsed time to break the hash:  0.5781536102294922


In a Windows 10 machine with 16 GB RAM, it takes around 0.55 seconds to break the password. 

**BONUS**: Modify the code to be able to crack hashes created with passwords more than one character long.

In [ ]:
# Use this cell to write your improved code
import itertools

list_of_saltchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./"
list_of_passchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./\-+%@,*"

def generateWords(size):
    return [''.join(word) for word in itertools.product(list_of_passchars, repeat=size)]

def bruteForceImpr(hashtobreak):
# 2. Iterate the list of salt chars using two fors so that you can test all possible salt combinations
    for k in range(1, 3):
        words = generateWords(k)
        #print(words)
        for i in list_of_saltchars:
            for j in list_of_saltchars:
                salt = i+j
                # 3. For all characters in list_of_passchars, hash with the salt
                for word in words:
                    attempt = des_crypt.hash(word, salt=salt)
                    # 4. If there is a match, exit and return the charcter
                    if attempt == hashtobreak:
                        # 5. Return the broken password and the broken salt
                        return word, salt

deshash4 = des_crypt.hash("C+6", salt="0x")
deshash4
t = time.time()
brokenhash, brokensalt = bruteForceImpr(deshash4)
print('The password is:', brokenhash)
print('The salt is:', brokensalt)
print('Elapsed time to break the hash: ', time.time() - t)

In [ ]:
import threading
from threading import Event, Thread
import itertools

list_of_saltchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./"
list_of_passchars = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ./\-+%@,*"
found = False
ready = Event()

def generateWords(size):
    return [''.join(word) for word in itertools.product(list_of_passchars, repeat=size)]

def bruteForceIt(words, hashtobreak):
    global found
    global ready
    
    for i in list_of_saltchars:
        for j in list_of_saltchars:
            salt = i+j
            # 3. For all characters in list_of_passchars, hash with the salt
            for word in words:
                attempt = des_crypt.hash(word, salt=salt)
                # 4. If there is a match, exit and return the charcter
                if found == True:
                    #print("Thread terminated!")
                    return False
                if attempt == hashtobreak:
                    # 5. Return the broken password and the broken salt
                    print('The password is:', word)
                    print('The salt is:', salt)
                    found = True
                    ready.set()
                    return True

def bruteForcePool(hashtobreak):
# 2. Iterate the list of salt chars using two fors so that you can test all possible salt combinations
    
    results = []
    for i in range(1, 4):
        words = generateWords(i)
        threading.Thread(target=bruteForceIt, args=(words, hashtobreak)).start()

deshash4 = des_crypt.hash("0000", salt="00")
t = time.time()
bruteForcePool(deshash4)
ready.wait()
print('Elapsed time to break the hash: ', time.time() - t)